<a href="https://colab.research.google.com/github/ImNotDanish05/2025_PBO_TI-1A/blob/main/APIMaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Free - Nominatim API (Open Sorce, Free, No Login at All)**
Use big data of OpenStreetMap

In [40]:
import requests

lat, lng = -6.2088, 106.8456
radius = 1000  # meter
query = "hospital"

print(f"Mencari lokasi di sekitar: {lat}, {lng} dengan radius {radius} meter...")

url = "https://nominatim.openstreetmap.org/search"

params = {
    "q": query,
    "format": "json",
    "limit": 5,
    "viewbox": f"{lng - 0.01},{lat - 0.01},{lng + 0.01},{lat + 0.01}",
    "bounded": 1
}

headers = {
    "User-Agent": "DanishSenpaiApp/1.0 (your@email.com)"
}

try:
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()

    if data:
        print("Hasil pencarian:")
        for place in data:
            print(f"{place['display_name']} (lat: {place['lat']}, lon: {place['lon']})")
    else:
        print("Tidak ada hasil ditemukan.")

except Exception as e:
    print(f"Terjadi kesalahan saat membuat permintaan: {e}")


Mencari lokasi di sekitar: -6.2088, 106.8456 dengan radius 1000 meter...
Hasil pencarian:
Rumah Sakit Khusus THT dan Bedah Kepala Leher Proklamasi, 43, Jalan Proklamasi, RW 02, Pegangsaan, Menteng, Jakarta Pusat, Daerah Khusus ibukota Jakarta, Jawa, 10320, Indonesia (lat: -6.2017111, lon: 106.8474650)
RS Agung, Jalan Padang, RW 08, Pasar Manggis, Setiabudi, Jakarta Selatan, Daerah Khusus ibukota Jakarta, Jawa, 12850, Indonesia (lat: -6.2092297, lon: 106.8463840)
RS K, Jalan Proklamasi, RW 02, Pegangsaan, Menteng, Jakarta Pusat, Daerah Khusus ibukota Jakarta, Jawa, 10320, Indonesia (lat: -6.2020722, lon: 106.8468531)
RS Jantung Jakarta, Jalan Matraman, RW 01, Palmeriam, Matraman, Jakarta Timur, Daerah Khusus ibukota Jakarta, Jawa, 13140, Indonesia (lat: -6.2001958, lon: 106.8553573)
RSIA Tambak, Jalan Tambak Lorong Buntu, RW 06, Pegangsaan, Menteng, Jakarta Pusat, Daerah Khusus ibukota Jakarta, Jawa, 10320, Indonesia (lat: -6.2063911, lon: 106.8483529)


In [42]:
# @markdown Lets go make a map hehehe >:D
import folium
from IPython.display import display

# Buat peta centered di lat, lng
m = folium.Map(location=[lat, lng], zoom_start=14)

# Tambahkan marker untuk setiap hasil
for place in data:
    name = place["display_name"]
    place_lat = float(place["lat"])
    place_lon = float(place["lon"])

    folium.Marker(
        [place_lat, place_lon],
        popup=name,
        tooltip="Klik untuk lihat nama"
    ).add_to(m)

# Langsung tampilkan di output Google Colab
display(m)


# **Not Free - FourSquare API (Only Trial, MUST HAVE CREDIT CARD)**

In [31]:
# @title Config
# @markdown WARNING! This API IS NOT FREE, DO NOT USE IT UNLESS YOU RICH:
# @markdown https://foursquare.com/developers/

from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Danish05/Account/foursquare.com.txt'

with open(file_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        if "Client Id:" in line:
            YOUR_CLIENT_ID = line.split("Client Id:")[1].strip()
        if "Client Secret:" in line:
            YOUR_CLIENT_SECRET = line.split("Client Secret:")[1].strip()
        if "Service API Key:" in line:
            YOUR_SERVICE_API_KEY = line.split("Service API Key:")[1].strip()
        if "Additional API Key:" in line:
            YOUR_ADDITIONAL_API_KEY = line.split("Additional API Key:")[1].strip()

CLIENT_ID = YOUR_CLIENT_ID # @param
CLIENT_SECRET = YOUR_CLIENT_SECRET # @param
SERVICE_API_KEY = YOUR_SERVICE_API_KEY # @param
ADDITIONAL_API_KEY = YOUR_ADDITIONAL_API_KEY # @param

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import requests
import json

# --- Koordinat Lokasi Anda ---
# GANTI DENGAN KOORDINAT RUMAH ANDA (latitude, longitude)
# Contoh: Koordinat Jakarta Pusat
LATITUDE = -6.2088
LONGITUDE = 106.8456

# --- Parameter Pencarian ---
RADIUS = 1000  # Radius pencarian dalam meter (misalnya 1000 meter = 1 km)
LIMIT = 50     # Jumlah hasil yang ingin diambil

# URL endpoint Foursquare Places API (v2)
URL = 'https://api.foursquare.com/v2/venues/explore'

# Parameter permintaan
params = {
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'll': f"{LATITUDE},{LONGITUDE}",
    'v': '20230303',  # Tanggal versi API (gunakan tanggal saat ini atau terbaru)
    'radius': RADIUS,
    'limit': LIMIT
}

print(f"Mencari lokasi di sekitar: {LATITUDE}, {LONGITUDE} dengan radius {RADIUS} meter...")

try:
    response = requests.get(URL, params=params)
    response.raise_for_status()  # Akan memunculkan HTTPException untuk respons status buruk (4xx atau 5xx)

    data = response.json()

    # Periksa jika ada grup item
    if 'response' in data and 'groups' in data['response']:
        venues = data['response']['groups'][0]['items']

        if venues:
            print(f"\nDitemukan {len(venues)} lokasi:")
            for venue in venues:
                name = venue['venue']['name']
                category = venue['venue']['categories'][0]['name'] if venue['venue']['categories'] else 'N/A'
                address = venue['venue']['location']['address'] if 'address' in venue['venue']['location'] else 'N/A'
                distance = venue['venue']['location']['distance']
                print(f"- Nama: {name}, Kategori: {category}, Alamat: {address}, Jarak: {distance}m")
        else:
            print("\nTidak ada lokasi yang ditemukan di sekitar koordinat yang diberikan.")
    else:
        print("\nTidak ada data yang sesuai dalam respons API.")
        print(json.dumps(data, indent=4)) # Cetak respons lengkap untuk debugging

except requests.exceptions.RequestException as e:
    print(f"Terjadi kesalahan saat membuat permintaan: {e}")
    if response:
        print(f"Status Kode: {response.status_code}")
        print(f"Pesan Kesalahan Foursquare: {response.text}")
except KeyError as e:
    print(f"Terjadi kesalahan saat memproses data JSON: Kunci tidak ditemukan - {e}")
    print(json.dumps(data, indent=4)) # Cetak data yang diterima untuk debugging

Mencari lokasi di sekitar: -6.2088, 106.8456 dengan radius 1000 meter...
Terjadi kesalahan saat membuat permintaan: 402 Client Error: Payment Required for url: https://api.foursquare.com/v2/venues/explore?client_id=K1FTHTAAQVC4UNRU1FKVWDJMFBWDSWEWRK5WZSXLXDNUBRDK&client_secret=LSRWMEFBYP1ZZU0I5VB4KJFEJW31XPJNCP5MWPKXRROASL5B&ll=-6.2088%2C106.8456&v=20230303&radius=1000&limit=50


# **Not Free - Google Maps API (Free, but NEED CREDIT CARD. MEANING STILL PAY TAXES ;-;)**

In [36]:
# @title Config
# @markdown WARNING! This API IS NOT FREE, DO NOT USE IT UNLESS YOU RICH:
# @markdown https://console.cloud.google.com/
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Danish05/Account/google/PrivateKeyAPI.txt'

with open(file_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        if "PrivateKey:" in line:
            YOUR_PRIVATE_KEY = line.split("PrivateKey:")[1].strip()

PRIVATE_KEY = YOUR_PRIVATE_KEY # @param

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import requests


lat, lng = -6.966667, 110.416664
radius = 500
query = "hospital"

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "location": f"{lat},{lng}",
    "radius": radius,
    "keyword": query,
    "key": PRIVATE_KEY
}

response = requests.get(url, params=params)
print("Status code:", response.status_code)

data = response.json()
print("Response JSON:")
print(data)


Status code: 200
Response JSON:
{'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
